In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt

% matplotlib inline
plt.style.use('ggplot')

# Loading MNIST Data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNISR_data", one_hot = True)

In [ ]:
print('Train Set Size: {}' .format(len(mnist.train.images)))
print('Validation Set Size: {}' .format(len(mnist.validation.images)))
print('Test Set Size: {}' .format(len(mnist.test.images)))

# Print an Random Example

In [ ]:
randi = np.random.randint(len(mnist.train.images))
plt.imshow(mnist.train.images[randi].reshape(28,28))
for i in range(0,9):
    if mnist.train.labels[randi][i]==1:
        print('Label: {}' .format(i))

# Weight and Bias Initalisation 

In [ ]:
def weight_init(shape):
    initial= tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_init(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Convolution and Pooling


In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')

def max_pooling_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Deep Network

In [ ]:
# Conv > pool > conv > pool > dense > dropout > out 

def deepnn(x): 
    '''
    x: input tensor with dimensions (n, 784)
    
    returns (y, keep_prob)
    y: tensor of shape(n, 10)
    keep_prob: scalar placeholder for the probability of dropout
    '''
    
    # Reshape for convolution
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # First Convolutional Layer
    W_conv1 = weight_init([5,5,1,32])
    b_conv1 = bias_init([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    
    # First Pooling Layer 
    h_pool1 = max_pooling_2x2(h_conv1)
    
    # Second Convolutional Layer
    W_conv2 = weight_init([5,5,32,64])
    b_conv2 = bias_init([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
    
    # Second Pooling Layer 
    h_pool2 = max_pooling_2x2(h_conv2)
    
    
    # First Fully Connected Layer - after 2 round of downsampling, our 28x28 image
    # is down to 7x7x64 feature maps -- maps this to 1024 features.
    W_fc1 = weight_init([7*7*64, 1024])
    b_fc1 = bias_init([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    # Dropout 
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    
    # Second FC, Output 
    W_fc2 = weight_init([1024, 10])
    b_fc2 = bias_init([10])
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) +  b_fc2
    return y_conv, keep_prob

# Building the Model


In [ ]:
image_shape = 28
input_size = image_shape * image_shape
output_size =10

learning_rate = 0.001
batch_size = 150

epoch = 20
epoch_size = 100
n_steps = epoch * epoch_size 

# Create / Reset the graph
tf.reset_default_graph()

# Input Images and Target Output Classes
x = tf.placeholder(tf.float32, shape=[None, input_size])
y_ = tf.placeholder(tf.float32, shape=[None, output_size])

y_conv, keep_prob = deepnn(x)

# Note: tf.nn.softmax_cross_entropy_with_logits internally applies the softmax on the model's unnormalised model prediction and sums across all calsses 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Train 
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

# Measurements 
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Training Session


In [ ]:
%%time

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for step in range(n_steps+1):
    # Training
    batch = mnist.train.next_batch(batch_size)
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    # Print outs
    if step%epoch_size==0:
             print("Epoch: {} Accuracy: {:4.2f} " .format(int(step/epoch_size),
                accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob: 1})))

# Predication Accuracy

In [ ]:
print('Test Accuracy: {:4.2f}%' .format(100*accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels})))